# Init

This is a init file with all the imports, functions and dataframes that are necessary.

**Change path here!**

In [1]:
## Path for the downloaded folder

PATH = '/Users/luanareis/Documents/IC/CTA_BrighteningAnalysis'

In [2]:
## Path for the folder with the downloaded lightcurves from the catalog

PATH_3DAY_LC = f'{PATH}/lightcurve_downloader/downloaded_lightcurves/3day_lightcurves'
PATH_WEEKLY_LC = f'{PATH}/lightcurve_downloader/downloaded_lightcurves/weekly_lightcurves'

Printing the paths to help using it in another files.

In [3]:
print('When you run __init__.ipynb, you upload the imports, as well as the paths, functions and dataframes necessary for the analysis.')

print('\n################### IMPORTANT PATHS ###################')

print('\n--> PATH : for the main folder CTA_BrighteningAnalysis downloaded from Github')

print('\nFor the downloaded lightcurves from the catalog folder, use:')

print('-> PATH_3DAY_LC : 3-day bin lightcurves')
print('-> PATH_WEEKLY_LC : weekly bin lightcurves')


When you run __init__.ipynb, you upload the imports, as well as the paths, functions and dataframes necessary for the analysis.

################### IMPORTANT PATHS ###################

--> PATH : for the main folder CTA_BrighteningAnalysis downloaded from Github

For the downloaded lightcurves from the catalog folder, use:
-> PATH_3DAY_LC : 3-day bin lightcurves
-> PATH_WEEKLY_LC : weekly bin lightcurves


Imports

In [4]:
## Astro imports

from astropy.io import fits
import numpy as np
from astropy.table import QTable
import astropy.units as u
from astropy.io import ascii
import os
import json
import urllib.request

from astropy import units as u

from astropy.time import Time,TimeUnix
from datetime import datetime

In [5]:
## Other imports

import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as sp
import scipy.odr.odrpack as odrpack
from scipy import signal, integrate
from scipy.fft import fft, fftfreq
import numpy as np
import statistics
import csv
import math
from scipy.fft import fft, fftfreq
import glob

import matplotlib.ticker as mticker
from matplotlib.ticker import FormatStrFormatter

## Functions

- convert_MET_UTC: converts the x-axis array of the lightcurve from MET to UTC - https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data/Time_in_ScienceTools.html

- dNdE: log-normal representation (LogParabola under SpectrumType in the FITS table) and a simple power-law form for $\beta = 0$. - https://arxiv.org/pdf/1902.10045.pdf

$$ \frac{dN}{dE} =  K\left ( \frac{E}{E_{0}} \right )^{-\alpha \ - \beta \ log(E/E_{0})} $$

- diff_flux: Differential Flux $\nu F_{\nu }$, particle flux density per unit energy incident on a surface $[erg\ cm^{-2} s^{-1}]$.

$$ \nu F_{\nu } = E^{2} \  \frac{dN}{dE} = E^{2} \  K\left ( \frac{E}{E_{0}} \right )^{-\alpha \ - \beta \ log(E/E_{0})} $$

- extract_data_from_file: given the bins (Weekly/ 3-days), returns the extracted data from the .json file, returning many arrays.

- synchr_type_dataframe: given the file_name, returns the dataframe that the file is contained based on its Synchroton Type.

- extract_parameters_from_table: parameters from FITS table of 10-year observations $ [0.1-100\ GeV\ ph\ cm^{−2}\ s^{−1}] $, by using


$$ \int \frac{dN}{dE} dE \Rightarrow  \left [ \int \frac{dN}{dE} dE \right ] = \frac{ph}{cm^{2} \ s}[0.1-100 \ GeV] $$


- relative_difference: relative difference between the flux and the spectrum flux

$$ relative.difference = \frac{ \left | \ spectrum.flux - average.flux \ \right | }{ average.flux } $$

- filter_percentage: filtering the sources with a % difference between the average and the spectrum flux, if the relative differece in a percentage is smaller than the chosen value, select it when True.

-  plot_SED: plot of the Spectral Energy Distribution, the differential flux $\nu\ F_{\nu}$ in units of '$erg \ cm^{-2} \ s^{-1}$' as a function of the Energy in '$GeV$'.

- plot_lc: plot of the LightCurve, the Photon Flux in units of '$ 0.1 - 100 \ GeV \ ph \ cm^{-2} \ s^{-1}$' as a fuction of Time in '$MET \ s$'. If you want to change it to UTC, you can use the function convert_MET_UTC to convert the time array and change the name of the x-axis :)

- plot_histogram: make Histograms of the flux_proportion of the Flux/Spectrum.Flux for each flux point in each lightcurve of the catalog in a y log scale.

In [6]:

## ---------- convert time from MET to UTC ----------
def convert_MET_UTC(time_MET):
    ########## Description ##########
    # more info on time in LAT:
    # https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data/Time_in_ScienceTools.html
    # ---------- Input ----------
    # time_MET : time using MET (seconds) from January 1st, 2001 until now  (1D-array)
    # ---------- Output ----------
    # time in UTC                                                           (1D-array)
    #################################
    
    # changing data to Unix (referencial of 1970)
    time_Unix = Time(time_MET, format='unix', scale='utc')

    # finding how many seconds are there from 1970 to January 1st, 2001
    time_difference = Time('2001-01-01', format='iso', scale='utc')
    time_difference.format = 'unix'
    time_difference.value

    # increased the diffence of time between 2001 and 1970 to my data
    time_MET_copy = np.copy(time_MET)
    time_MET_copy += time_difference.value

    # transform into year: format='iso'
    time_Unix = Time(time_MET_copy, format='unix', scale='utc')
    time_Unix.format = 'iso'
    time_Unix

    # list with only date values, year, month and day
    time_UTC = []
    for i in range(len(time_Unix.value)):
        time_UTC.append(datetime.strptime(time_Unix.value[i][:10], '%Y-%m-%d'))

    return time_UTC




## ---------- log-normal representation ----------
def dNdE(E, E_0, K, alpha, beta=None):
    ########## Description ##########
    # ---------- Input ----------
    # E     : value of the energy                               (1D-array)
    # E_0   : value of the pivot_Energy in erg                  (float)
    # K     : value of the flux_Density in erg-1 cm-2 s-1       (float)
    # alpha : Index                                             (float)
    # beta  : Beta (only for LogParabola SED_class)             (float)
    # ---------- Output ----------
    # log-normal representation value dN/dE in 'erg-1 cm-2 s-1' (1D-array)
    #################################
    
    if (beta == None):
        Y = K * ((E/E_0)**(-alpha))
    else:
        Y = K * ((E/E_0)**(-alpha - beta * np.log(E/E_0)))       
    return Y




## ---------- Differential Flux ----------
def diff_flux(E, E_0, K, alpha, beta=None):
    ########## Description ##########
    # ---------- Input ----------
    # E     : value of the energy                          (1D-array)
    # E_0   : value of the pivot_Energy in erg             (float)
    # K     : value of the flux_Density in erg-1 cm-2 s-1  (float)
    # alpha : Index                                        (float)
    # beta  : Beta (only for LogParabola SED_class)        (float)
    # ---------- Output ----------
    # differential flux value E^2 dN/dE in 'erg cm-2 s-1'  (1D-array)
    #################################
    
    if (beta == None):
        Y = E**2 * K * ((E/E_0)**(-alpha))
    else:
        Y = E**2 * K * ((E/E_0)**(-alpha - beta * np.log(E/E_0)))     
    return Y




## ---------- extract data from .json file ----------
def extract_data_from_file(bins, file_name):
    ########## Description ##########
    # ---------- Input ----------
    # file_name : name of .json file                          (string)
    # ---------- Output ----------
    # name : name of the source                               (string)
    # time, flux : axis for Light Curve                       (array)
    # time_error, flux_low_error, flux_high_error : error     (array)
    # time_upper_lim, flux_upper_lim : upper limits           (array)
    #################################
    
    if (bins == 'weekly'):
        file = open(f'{PATH_WEEKLY_LC}/{file_name}')
    if (bins == '3-days'):
        file = open(f'{PATH_3DAY_LC}/{file_name}')
    
    name = file_name[5:-5]
    
    # returns JSON object as a dictionary
    data = json.load(file)
    
    # creating arrays to store the data
    time = np.array(data['flux'])[:,0]     # 'flux' [i][0]
    flux = np.array(data['flux'])[:,1]     # 'flux' [i][1]
    
    time_error      = np.array(data['flux_error'])[:,0]     # 'flux_error' [i][0]
    flux_low_error  = np.array(data['flux_error'])[:,1]     # 'flux_error' [i][1]  - lower flux edge
    flux_high_error = np.array(data['flux_error'])[:,2]     # 'flux_error' [i][2]  - high edge
    
    time_upper_lim  = np.array(data['flux_upper_limits'])[:,0]     # 'flux_upper_limits' [i][0]
    flux_upper_lim  = np.array(data['flux_upper_limits'])[:,1]     # 'flux_upper_limits' [i][1]
    
    return name, time, flux, time_error, flux_low_error, flux_high_error, time_upper_lim, flux_upper_lim




## ---------- Which dataframe to use based on the Synchroton Type ----------
def synchr_type_dataframe(file_name):
    ########## Description ##########
    # ---------- Input ----------
    # file_name : name of the file in .json         (string)
    # ---------- Output ----------
    # dataframe : which dataframe to use            (dataframe)
    #################################
    
    ## ---------- Low Synchrotron Peak ----------
    if (dfLSP['Source_Name'] == file_name[5:-5]).any():
        dataframe = dfLSP

    ## ---------- Intermediate Synchrotron Peak ----------
    if (dfISP['Source_Name'] == file_name[5:-5]).any():
        dataframe = dfISP
        
    ## ---------- High Synchrotron Peak ----------
    if (dfHSP['Source_Name'] == file_name[5:-5]).any():
        dataframe = dfHSP
        
    return dataframe




## ---------- parameters from FITS table of 10-year observations ----------
def extract_parameters_from_table(name, dataframe, flux):
    ########## Description ##########
    # ---------- Input ----------
    # name : name of the source                                (string)
    # dataframe : which df to use                              (dataframe)
    # flux : flux of the source from .json file                (array)
    # ---------- Output ----------
    # E : energy                                               (array)
    # differential_flux : differential flux                    (array)
    # spec_type : spectrum type                                (string)
    # flux_from_spectrum : integrated flux from the SED        (float)
    #################################

    # defining Energy in 'erg'
    E = ((np.logspace(np.log10(0.1), np.log10(100), 100) * u.GeV).to('erg')).value     # log scale of Energy in erg

    # get the index of each one
    index = dataframe[dataframe['Source_Name'] == name].index

    for i in index:

        # Pivot_Energy in erg
        E0 = ((dataframe.loc[i,'Pivot_Energy'] * u.MeV).to('erg')).value

        ## ---------- PowerLaw ----------
        if dataframe.loc[i,'SpectrumType'] == 'PowerLaw         ':
            spec_type = "PowerLaw"

            # PL_Flux_Density in erg-1 cm-2 s-1
            K = ((dataframe.loc[i,'PL_Flux_Density'] * u.MeV**-1 * u.cm**-2 * u.s**-1).to('erg-1 cm-2 s-1')).value
            # PL_Index
            alpha = dataframe.loc[i,'PL_Index']

            differential_flux = diff_flux(E, E0, K, alpha)

            integrateflux = integrate.quad(lambda x: K * ((x/E0)**(-alpha)),
                                           (0.1*u.GeV).to('erg').value, (100*u.GeV).to('erg').value)

            flux_from_spectrum = integrateflux[0] # returning only the first value of integrateflux

        ## ---------- LogParabola ----------
        elif dataframe.loc[i,'SpectrumType'] == 'LogParabola      ':
            spec_type = "LogParabola"

            # LP_Flux_Density in erg-1 cm-2 s-1
            K = ((dataframe.loc[i,'LP_Flux_Density'] * u.MeV**-1 * u.cm**-2 * u.s**-1).to('erg-1 cm-2 s-1')).value
            # LP_Index
            alpha = dataframe.loc[i,'LP_Index']
            # LP_beta
            beta = dataframe.loc[i,'LP_beta']

            differential_flux = diff_flux(E, E0, K, alpha, beta)

            integrateflux = integrate.quad(lambda x: K * ((x/E0)**(-alpha - beta * np.log(x/E0))), 
                                          (0.1*u.GeV).to('erg').value, (100*u.GeV).to('erg').value)

            flux_from_spectrum = integrateflux[0] # returning only the first value of integrateflux

        ## ---------- in case there is an error ----------
        else:
            print('### error ###')
    
        
    return E, differential_flux, spec_type, flux_from_spectrum




## ---------- relative difference between the flux and spectrum flux ----------
def relative_difference(flux, flux_from_spectrum):
    ########## Description ##########
    # ---------- Input ----------
    # flux : the light curve flux                                                         (float)
    # flux_from_spectrum : integrated flux from the 10-years SED                          (float)
    # ---------- Output ----------
    # relative difference : relative difference between the average and integrated flux   (float)
    #################################
    
    average_flux = np.average(flux)
    
    rel_difference = np.abs(flux_from_spectrum - average_flux) / average_flux
    
    return rel_difference




## ---------- filtering the sources with a % difference between the average and spectrum flux ----------
def filter_percentage(name, flux, flux_from_spectrum, percentage):
    ########## Description ##########
    # ---------- Input ----------
    # name : name of the source                                     (string)
    # flux : the light curve flux                                   (float)
    # flux_from_spectrum : integrated flux from the 10-years SED    (float)
    # percentage : percentage we want to filter                     (float)
    # ---------- Output ----------
    # True / False (boolean) : if we include the source or not      (float)
    #################################
    
    rel_difference = relative_difference(flux, flux_from_spectrum) *100
    
    print(f'Source {name} percentage:', np.round(rel_difference))
    
    if (rel_difference <= percentage):
        return True
    else:
        return False




## ---------- plot Spectral Energy Distribution ----------
def plot_SED(E, differential_flux, power, name, spec_type, x_dlim, x_ulim, y_dlim, y_ulim):
    ########## Description ##########
    # ---------- Input ----------
    # E : energy created by logspace from 0.1 to 100 GeV,
    # given only the value in 'erg'                             (array)
    # differential_flux : differential flux                     (array)
    # power : power of the differential_flux                    (int)
    # name  : name of the source                                (string)
    # spect_type : Spectrum Type (logP/ powL)                   (string)
    # x_dlim, x_ulim : limits of x axis                         (float)
    # y_dlim, y_ulim : limits of y axis                         (float)
    # ---------- Output ----------
    # 
    #################################

    plt.figure(figsize=(7,5), dpi=100)

    plt.plot(((E*u.erg).to('GeV')).value, differential_flux*10**(power), '+', markersize=2, color='black')
    plt.plot(((E*u.erg).to('GeV')).value, differential_flux*10**(power), '--', linewidth=0.4, color='black')

    plt.xscale('log')
    plt.yscale('log')
    plt.grid()

    plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    plt.xlim(x_dlim, x_ulim)
    plt.ylim(y_dlim, y_ulim)

    
    plt.title(f'4FGL+{name} - {spec_type} Spectrum', fontsize='large')
    plt.ylabel(r'$\nu\ F_{\nu}$ [ $ 10^{- %d } $ erg $cm^{-2}$ $s^{-1}$]' % (power), fontsize=12)
    plt.xlabel('Energy (GeV)', fontsize=12)
    
    return




## ---------- plot LightCurve ----------
def plot_lc(name, time, flux, time_error, flux_low_error, flux_high_error,
            time_upper_lim, flux_upper_lim, ylim, flux_from_spectrum):
    ########## Description ##########
    # ---------- Input ----------
    # name: name of the source                                (string)
    # time, flux : axis for Light Curve                       (array)
    # time_error, flux_low_error, flux_high_error : error     (array)
    # time_upper_lim, flux_upper_lim : upper limits           (array)
    # ylim: limit of the y axis                               (float)
    # flux_from_spectrum : integrated flux over 10-year SED   (float)
    # ---------- Output ----------
    # 
    #################################
    
    difference = relative_difference(flux, flux_from_spectrum)
    print('difference:', difference)
    
    plt.figure(figsize=(17,5), dpi=300)

    # flux
    plt.plot(time, flux, '.', markersize=10, label='Flux Points')
    plt.plot(time, flux, linewidth=0.4, color='black')

    # upper_limits
    plt.plot(time_upper_lim, flux_upper_lim, 'v', color='gray', markersize=3, alpha=0.45, label='Upper Limits')

    # error_bar
    # plt.plot(time_error, flux_low_error, 'x', color='green', markersize=3, alpha=0.3)  # visualize the error
    # plt.plot(time_error, flux_high_error, 'x', color='red', markersize=3, alpha=0.3)   # visualize the error
    plt.errorbar(time, flux, yerr=flux_high_error-flux, linewidth=0.2, color='black', alpha=0.9)
    plt.errorbar(time, flux, yerr=flux-flux_low_error, linewidth=0.2, color='black', alpha=0.9)

    # spectrum flux
    plt.hlines(y=flux_from_spectrum, xmin=np.min(time), xmax=np.max(time), linewidth=2, linestyles='dashed', color='r', label='Flux from Spectrum')
    plt.hlines(y=np.average(flux), xmin=np.min(time), xmax=np.max(time), linewidth=2, linestyles='dashed', color='b', label='Average Flux')

    # legend
    plt.legend() # f'difference: {difference}'
    
    plt.ylim(0, ylim)
    plt.title(f'4FGL+{name} Light Curve', fontsize='xx-large')
    plt.ylabel('Photon Flux (0.1-100 GeV ph $cm^{-2}$ $s^{-1}$)', fontsize=12)
    plt.xlabel('Time (MET s)', fontsize=12) # Date (UTC)
    
    return




## ---------- make Histograms ----------
def plot_histogram(flux_proportion, bins1, bins2, x_low_lim, x_high_lim):
    ########## Description ##########
    # ---------- Input ----------
    # flux_proportion : flux / integrated flux                 (array)
    # bins1: name of the source                                (int)
    # bins2 : axis for Light Curve                             (int)
    # x_low_lim: inferior limit of x axis                      (float)
    # x_high_lim: superior limit of x axis                     (float)
    # ---------- Output ----------
    # 
    #################################

    n, bins, patches = plt.hist(flux_proportion, bins1, density=True, facecolor='g', alpha=0.75)
    n, bins, patches = plt.hist(flux_proportion, bins2, density=True, facecolor='red', alpha=0.75)

    plt.xlabel('Flux / Spectrum Flux')
    plt.ylabel('Frequency')

    plt.xlim(x_low_lim, x_high_lim)
    plt.yscale('log')

    plt.grid(True)
    plt.show()
    
    return

## Dataframes

In [7]:
## ---------- Low Synchrotron Peak ----------
dfLSP = pd.read_csv(f'{PATH}/4LAC_catalog_generator_v2/LSP_data.csv')
## ---------- Intermediate Synchrotron Peak ----------
dfISP = pd.read_csv(f'{PATH}/4LAC_catalog_generator_v2/ISP_data.csv')
## ---------- High Synchrotron Peak ----------
dfHSP = pd.read_csv(f'{PATH}/4LAC_catalog_generator_v2/HSP_data.csv')

## ---------- Whole Dataframe ----------
dat = pd.read_csv(f'{PATH}/4LAC_catalog_generator_v2/whole_dataframe.csv')

In [8]:
print('\n##################### DATAFRAMES ######################')

print('\n# of sources in the dataframes:')
print('dfLSP:', len(dfLSP))
print('dfISP:', len(dfISP))
print('dfHSP:', len(dfHSP))
print('---> total:', len(dfLSP) + len(dfISP) + len(dfHSP))

print('\ndat:', len(dat))


##################### DATAFRAMES ######################

# of sources in the dataframes:
dfLSP: 1538
dfISP: 508
dfHSP: 548
---> total: 2594

dat: 3511


## Filtering the .json files that are on each table (filter by Synchrotron Type)

### 3 days bin

In [9]:
## Select every file that ends with .json in folder
lc_3day = glob.glob(PATH_3DAY_LC + '/*.json')

## Create empty list to store dataframes
lc_downloaded_3days = []

## Loop through list of files and read each one into a dataframe and append to list
for file in lc_3day:
    
    ## Removing the path from the file name
    stripped_file = file.lstrip(f'{PATH_3DAY_LC}/')
    
    ## Append to list of the lightcurves
    lc_downloaded_3days.append(stripped_file)

In [10]:
## Create a list for each Synchrotron Type
lc_downloaded_LSP_3days = [] # low
lc_downloaded_ISP_3days = [] # intermediate
lc_downloaded_HSP_3days = [] # high

for file in lc_downloaded_3days:
    ## ---------- Low Synchrotron Peak ----------
    if (dfLSP['Source_Name'] == file[5:-5]).any():
        lc_downloaded_LSP_3days.append(f'4FGL+{file[5:-5]}.json')
        
    ## ---------- Intermediate Synchrotron Peak ----------
    if (dfISP['Source_Name'] == file[5:-5]).any():
        lc_downloaded_ISP_3days.append(f'4FGL+{file[5:-5]}.json')
        
    ## ---------- High Synchrotron Peak ----------
    if (dfHSP['Source_Name'] == file[5:-5]).any():
        lc_downloaded_HSP_3days.append(f'4FGL+{file[5:-5]}.json')

In [11]:
print('\n############### LIGHTCURVES FROM FOLDER ###############')

print('\n--------------- 3-day bin ---------------')

print('\n# of lightcurves from the folder in the array:')
print('lc_downloaded_LSP_3days:', len(lc_downloaded_LSP_3days))
print('lc_downloaded_ISP_3days:', len(lc_downloaded_ISP_3days))
print('lc_downloaded_HSP_3days:', len(lc_downloaded_HSP_3days))
print('---> total:', len(lc_downloaded_LSP_3days) + len(lc_downloaded_ISP_3days) + len(lc_downloaded_HSP_3days))

print('\nlc_downloaded_3days:', len(lc_downloaded_3days))


############### LIGHTCURVES FROM FOLDER ###############

--------------- 3-day bin ---------------

# of lightcurves from the folder in the array:
lc_downloaded_LSP_3days: 465
lc_downloaded_ISP_3days: 60
lc_downloaded_HSP_3days: 66
---> total: 591

lc_downloaded_3days: 683


To order the numbers correctly

In [12]:
lc_downloaded_HSP_3days.sort()
# lc_downloaded_HSP_3days

In [13]:
lc_downloaded_ISP_3days.sort()
# lc_downloaded_ISP_3days

In [14]:
lc_downloaded_LSP_3days.sort()
# lc_downloaded_LSP_3days

### Weekly bin

In [15]:
## Select every file that ends with .json in folder
lc_weekly = glob.glob(PATH_WEEKLY_LC + '/*.json')

## Create empty list to store dataframes
lc_downloaded_weekly = []

## Loop through list of files and read each one into a dataframe and append to list
for file in lc_weekly:
    
    ## Removing the path from the file name
    stripped_file = file.lstrip(f'{PATH_WEEKLY_LC}/')
    
    ## Append to list of the lightcurves
    lc_downloaded_weekly.append(stripped_file)

In [16]:
## Create a list for each Synchrotron Type
lc_downloaded_LSP_weekly = [] # low
lc_downloaded_ISP_weekly = [] # intermediate
lc_downloaded_HSP_weekly = [] # high

for file in lc_downloaded_weekly:
    ## ---------- Low Synchrotron Peak ----------
    if (dfLSP['Source_Name'] == file[5:-5]).any():
        lc_downloaded_LSP_weekly.append(f'4FGL+{file[5:-5]}.json')
        
    ## ---------- Intermediate Synchrotron Peak ----------
    if (dfISP['Source_Name'] == file[5:-5]).any():
        lc_downloaded_ISP_weekly.append(f'4FGL+{file[5:-5]}.json')
        
    ## ---------- High Synchrotron Peak ----------
    if (dfHSP['Source_Name'] == file[5:-5]).any():
        lc_downloaded_HSP_weekly.append(f'4FGL+{file[5:-5]}.json')

In [17]:
## Making sure we have the right count

print('\n--------------- Weekly bin ---------------')

print('\n# of lightcurves from the folder in the array:')
print('lc_downloaded_LSP_weekly:', len(lc_downloaded_LSP_weekly))
print('lc_downloaded_ISP_weekly:', len(lc_downloaded_ISP_weekly))
print('lc_downloaded_HSP_weekly:', len(lc_downloaded_HSP_weekly))
print('---> total:', len(lc_downloaded_LSP_weekly) + len(lc_downloaded_ISP_weekly) + len(lc_downloaded_HSP_weekly))

print('\nlc_downloaded_weekly:', len(lc_downloaded_weekly))


--------------- Weekly bin ---------------

# of lightcurves from the folder in the array:
lc_downloaded_LSP_weekly: 465
lc_downloaded_ISP_weekly: 60
lc_downloaded_HSP_weekly: 66
---> total: 591

lc_downloaded_weekly: 683


To order the numbers correctly

In [18]:
lc_downloaded_HSP_weekly.sort()
# lc_downloaded_HSP_weekly

In [19]:
lc_downloaded_ISP_weekly.sort()
# lc_downloaded_ISP_weekly

In [20]:
lc_downloaded_LSP_weekly.sort()
# lc_downloaded_LSP_weekly